In [3]:
from gan_u import *
import numpy as np
import os
import sys
sys.path.append("..")
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf

def classifier(in_shape=14):
	model = tf.keras.models.Sequential()
	model.add(tf.keras.layers.Dense(10, input_shape = (in_shape,), activation='relu'))
	model.add(tf.keras.layers.Dense(10, activation='relu'))
	model.add(tf.keras.layers.Dense(2, activation='softmax'))
	model.compile(optimizer='adam',
					  loss='sparse_categorical_crossentropy',
					  metrics=['accuracy'])	
	return model


def train_classifier(x,y,x_val=None,y_val=None,n_epochs=100):
	# Normal model building
	model_main = classifier()
	if x_val is None or y_val is None:
		history = model_main.fit(x, y, epochs = n_epochs)
	else:
		history = model_main.fit(x, y,validation_data=(x_val,y_val), epochs = n_epochs)
		# print("************training evaluation**********")
		# print(model_main.evaluate(x, y,verbose=0))
		# print('***********done**************')
	return model_main, history
	
df = pd.read_csv("./adult.data",header=None)
df = shuffle(df).reset_index(drop=True)

#scaler = StandardScaler()
scaler = MinMaxScaler()

LABEL = df.columns[-1]
df.dropna()
DF = df
for i in df.columns:
	DF = DF[DF[i]!=' ?']
df = DF

categorical=[2,4,6,7,8,9,10,14,15]

for i in categorical:  
	print(i)
	encoder = LabelEncoder()
	encoder.fit(df[i-1])
	df[i-1] = encoder.transform(df[i-1])


x = df.drop(LABEL, axis=1)


c = scaler.fit_transform(x)
x = pd.DataFrame(c)
X = x
Y = df[LABEL]
print(x)

X_Distributions = X
latent_dim=100
trainX, testX, trainY, testY = train_test_split(np.array(X), np.array(Y), test_size = 0.2, random_state = 0)

d_model = define_discriminator()
	# create the generator
g_model = define_generator(latent_dim)
	# create the gan
gan_model = define_gan(g_model, d_model)
	# load datset

i=0
n_samples = len(trainX)
	# train gan model
train(g_model, d_model, gan_model, trainX, latent_dim,n_epochs=100,client=i+1)

image_model, _ = train_classifier(trainX,trainY,n_epochs=100)
	# generate fake samples at the server
x_fake, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# classify fake samples
y_fake = image_model.predict_classes(x_fake)
y_fake = y_fake.reshape(-1,1)

	# generate fake samples at the server



print('Complete')

central_model, central_model_history = train_classifier(trainX,trainY,n_epochs=100)
combined_model, combined_model_history = train_classifier(x_fake,y_fake,x_val=trainX,y_val=trainY,n_epochs=100)

results = {}
results['FGAN testing accuracy(original_data)'] = combined_model.evaluate(testX,testY,verbose=0)[1]*100
results['Central model testing accuracy'] = central_model.evaluate(testX, testY, verbose=0)[1]*100


if not os.path.isdir('ganresults'):
	os.makedirs('ganresults')

results_save_path = 'ganresults/gan_distributed_results.txt'

with open(results_save_path, 'w') as f:

	print(combined_model_history.history['val_accuracy'],file=f)
	print('printing other results',file=f)
	print(results,file=f)
	print('******central model****')
	print(central_model_history.history['accuracy'],file=f)

x_axis= x_axis = [i for i in range(1,len(central_model_history.history['accuracy'])+1)]
plt.plot(x_axis,combined_model_history.history['val_accuracy'], label='FGAN')
plt.plot(x_axis,central_model_history.history['accuracy'],label='Central')

plt.title('models accuracy comparision')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='lower right')

name = 'ganresults/all_approaches_accuracy_vs_epochs'
plt.savefig(name+'.eps')
plt.savefig(name+'.png')
plt.clf()
print(results)

/anaconda/envs/py37_tensorflow/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


2
4
6
7
8
9
10
14
15
              0         1         2         3         4         5         6  \
0      0.493151  0.166667  0.110670  0.800000  0.866667  0.000000  0.692308   
1      0.260274  0.333333  0.042189  0.600000  0.800000  0.000000  0.000000   
2      0.479452  0.500000  0.084795  0.600000  0.800000  0.333333  0.230769   
3      0.547945  0.333333  0.149302  0.800000  0.866667  0.666667  0.230769   
4      0.246575  0.333333  0.164476  0.733333  0.533333  0.666667  0.538462   
...         ...       ...       ...       ...       ...       ...       ...   
30157  0.397260  0.500000  0.208272  0.733333  0.533333  0.333333  0.230769   
30158  0.342466  0.333333  0.118920  1.000000  0.600000  0.333333  0.153846   
30159  0.575342  0.166667  0.072639  0.733333  0.533333  0.833333  0.769231   
30160  0.561644  0.333333  0.115657  1.000000  0.600000  0.000000  0.000000   
30161  0.041096  0.333333  0.188135  0.733333  0.533333  0.666667  0.538462   

         7    8    9        10

/anaconda/envs/py37_tensorflow/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, 1/188, d=0.663, g=0.756
>1, 2/188, d=0.664, g=0.759
>1, 3/188, d=0.654, g=0.756
>1, 4/188, d=0.657, g=0.744
>1, 5/188, d=0.659, g=0.752
>1, 6/188, d=0.651, g=0.741
>1, 7/188, d=0.653, g=0.739
>1, 8/188, d=0.657, g=0.726
>1, 9/188, d=0.661, g=0.730
>1, 10/188, d=0.657, g=0.712
>1, 11/188, d=0.666, g=0.723
>1, 12/188, d=0.661, g=0.708
>1, 13/188, d=0.663, g=0.709
>1, 14/188, d=0.677, g=0.696
>1, 15/188, d=0.664, g=0.689
>1, 16/188, d=0.660, g=0.692
>1, 17/188, d=0.678, g=0.680
>1, 18/188, d=0.678, g=0.678
>1, 19/188, d=0.672, g=0.680
>1, 20/188, d=0.688, g=0.665
>1, 21/188, d=0.687, g=0.675
>1, 22/188, d=0.688, g=0.664
>1, 23/188, d=0.691, g=0.658
>1, 24/188, d=0.688, g=0.662
>1, 25/188, d=0.697, g=0.678
>1, 26/188, d=0.695, g=0.647
>1, 27/188, d=0.686, g=0.650
>1, 28/188, d=0.699, g=0.652
>1, 29/188, d=0.700, g=0.652
>1, 30/188, d=0.696, g=0.662
>1, 31/188, d=0.706, g=0.663
>1, 32/188, d=0.699, g=0.662
>1, 33/188, d=0.700, g=0.671
>1, 34/188, d=0.698, g=0.684
>1, 35/188, d=0.698, g=

>2, 106/188, d=0.693, g=0.662
>2, 107/188, d=0.695, g=0.668
>2, 108/188, d=0.695, g=0.667
>2, 109/188, d=0.694, g=0.669
>2, 110/188, d=0.693, g=0.674
>2, 111/188, d=0.691, g=0.674
>2, 112/188, d=0.690, g=0.680
>2, 113/188, d=0.692, g=0.675
>2, 114/188, d=0.687, g=0.680
>2, 115/188, d=0.693, g=0.681
>2, 116/188, d=0.695, g=0.687
>2, 117/188, d=0.694, g=0.690
>2, 118/188, d=0.693, g=0.689
>2, 119/188, d=0.695, g=0.696
>2, 120/188, d=0.692, g=0.693
>2, 121/188, d=0.696, g=0.698
>2, 122/188, d=0.695, g=0.699
>2, 123/188, d=0.698, g=0.706
>2, 124/188, d=0.692, g=0.704
>2, 125/188, d=0.690, g=0.704
>2, 126/188, d=0.689, g=0.705
>2, 127/188, d=0.690, g=0.707
>2, 128/188, d=0.693, g=0.705
>2, 129/188, d=0.699, g=0.711
>2, 130/188, d=0.693, g=0.705
>2, 131/188, d=0.692, g=0.703
>2, 132/188, d=0.691, g=0.704
>2, 133/188, d=0.694, g=0.706
>2, 134/188, d=0.694, g=0.702
>2, 135/188, d=0.691, g=0.701
>2, 136/188, d=0.692, g=0.701
>2, 137/188, d=0.690, g=0.701
>2, 138/188, d=0.694, g=0.701
>2, 139/18

...

755/755 [==============================] - 2s 3ms/step - loss: 0.0940 - accuracy: 0.9631 - val_loss: 0.8472 - val_accuracy: 0.8327
Epoch 14/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0930 - accuracy: 0.9636 - val_loss: 0.8504 - val_accuracy: 0.8326
Epoch 15/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0912 - accuracy: 0.9633 - val_loss: 0.8808 - val_accuracy: 0.8331
Epoch 16/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0894 - accuracy: 0.9641 - val_loss: 0.9416 - val_accuracy: 0.8344
Epoch 17/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0884 - accuracy: 0.9637 - val_loss: 0.9660 - val_accuracy: 0.8347
Epoch 18/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0873 - accuracy: 0.9646 - val_loss: 0.9648 - val_accuracy: 0.8346
Epoch 19/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0869 - accuracy: 0.9647 - val_loss: 0.9705 - val_accuracy: 0.8351
Epoch

Epoch 70/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0641 - accuracy: 0.9731 - val_loss: 1.3019 - val_accuracy: 0.8391
Epoch 71/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0619 - accuracy: 0.9742 - val_loss: 1.3186 - val_accuracy: 0.8384
Epoch 72/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0622 - accuracy: 0.9742 - val_loss: 1.3200 - val_accuracy: 0.8381
Epoch 73/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0617 - accuracy: 0.9741 - val_loss: 1.3752 - val_accuracy: 0.8381
Epoch 74/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0619 - accuracy: 0.9747 - val_loss: 1.2922 - val_accuracy: 0.8385
Epoch 75/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0630 - accuracy: 0.9740 - val_loss: 1.3603 - val_accuracy: 0.8394
Epoch 76/100
755/755 [==============================] - 2s 3ms/step - loss: 0.0612 - accuracy: 0.9752 - val_loss: 1.4191 - val_accuracy:

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


******central model****
{'FGAN testing accuracy(original_data)': 85.11520028114319, 'Central model testing accuracy': 84.86656546592712}


<Figure size 432x288 with 0 Axes>